In [4]:
!pip install qdrant-client 
!pip install sentence-transformers


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 943.1 kB/s eta 0:00:00 0:00:01
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Using cached pillow-11.3.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.0 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-

In [26]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

client = QdrantClient(host="localhost", port=6333)
collection_name = "my_collection"

client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

/var/folders/mt/ynd178zj0lsf9_dhk8rpqy200000gn/T/ipykernel_42251/4214217014.py:7: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [19]:
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

/var/folders/mt/ynd178zj0lsf9_dhk8rpqy200000gn/T/ipykernel_42251/1441803358.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [27]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
text = "Este es un ejemplo de registro"
vector = model.encode(text).tolist()

In [28]:
import numpy as np
from qdrant_client.models import PointStruct

point_id = 1
payload = {"text": text}

client.upsert(
    collection_name=collection_name,
    points=[
        PointStruct(id=point_id, vector=vector, payload=payload)
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [29]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

all_points = []
next_offset = None

while True:
    response = client.scroll(
        collection_name="my_collection",
        limit=100,
        offset=next_offset,
        with_payload=True,
        with_vectors=True
    )
    
    points, next_offset = response

    if not points:
        print("No more points.")
        break

    print(f"Fetched {len(points)} points")
    all_points.extend(points)

    # ✅ Prevent infinite loop: break if offset does not change
    if not next_offset:
        break

Fetched 1 points


In [34]:
query_vector = vector  # usa el mismo vector de ejemplo para probar

results = client.query_points(
    collection_name="my_collection",
    query=query_vector,
    limit=3
)

print(results)

points=[ScoredPoint(id=1, version=0, score=1.0000001, payload={'text': 'Este es un ejemplo de registro'}, vector=None, shard_key=None, order_value=None)]


In [36]:
points, _ = client.scroll(
    collection_name="my_collection",
    limit=100,
    with_payload=True,
    with_vectors=False  # solo necesitas el texto
)

for point in points:
    print(point.payload.get("text"))

Este es un ejemplo de registro
